In [1]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

## Model

In [2]:
import torch
from transformers import pipeline

model_name = "openai/whisper-medium"

model = pipeline(
    "automatic-speech-recognition", 
    model=model_name,
    torch_dtype=torch.float16, 
    device="mps"
)

Device set to use mps


## Files

In [3]:
from tqdm.notebook import tqdm

import importlib
import pandas as pd
import get_files as gf

In [4]:
importlib.reload(gf)

<module 'get_files' from '/Users/salgu/Workspace/1_project/cow/get_files.py'>

In [5]:
directory_path = "./audio"
extensions_to_find = ['.mp3']

grouped_files = gf.get_files_grouped_by_first_subdirectory(
    directory_path, 
    extensions_to_find
)

## Process

In [8]:
for first_subdir, files in tqdm(grouped_files.items()):
    print(f"Working Directory: {first_subdir}")
    transcriptions = []
    for file in tqdm(files):
        transcription = model(
            file,
            return_timestamps=True
        )
        transcriptions.append(transcription)
    df = gf.create_dataframe_from_list(transcriptions)
    df.to_csv(f"transcriptions_{first_subdir}.csv", index=False)

  0%|          | 0/3 [00:00<?, ?it/s]

Working Directory: 2022


  0%|          | 0/2 [00:00<?, ?it/s]

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


Working Directory: 2023


  0%|          | 0/2 [00:00<?, ?it/s]

Working Directory: 2024


  0%|          | 0/4 [00:00<?, ?it/s]

## Result

In [9]:
sample_df = pd.read_csv("transcriptions_2024.csv")
sample_df.head()

,text,timestamp
0,싸울 준비를 하고 있죠 자 양전 제2경기 이렇게 마무리가 됐습니다. 이 경기의 정...,"0.0 - 5.84: 싸울 준비를 하고 있죠, 7.08 - 9.68: 자 양전,..."
1,싸울 준비를 하고 있죠 자 양전 제2경기 이렇게 마무리가 됐습니다. 이 경기의 정...,"0.0 - 5.84: 싸울 준비를 하고 있죠, 7.08 - 9.68: 자 양전,..."
2,싸울 준비를 하고 있죠 자 양전 제2경기 이렇게 마무리가 됐습니다. 이 경기의 정...,"0.0 - 5.84: 싸울 준비를 하고 있죠, 7.08 - 9.68: 자 양전,..."
3,싸울 준비를 하고 있죠 자 양전 제2경기 이렇게 마무리가 됐습니다. 이 경기의 정...,"0.0 - 5.84: 싸울 준비를 하고 있죠, 7.08 - 9.68: 자 양전,..."
